In [1]:
import plotly.express as px
import pandas as pd
import geopandas as gpd
import json
from shapely.geometry import Point
from pprint import pprint
import requests
import os
import glob
from tqdm import tqdm

In [2]:
data = pd.read_pickle('data/preprocessed/accidents_in_london.pkl')
london_poly = gpd.read_file(json.load(open('data/london_poly.json')))
data['lat'] = data['geom'].apply(lambda x: x.coords[0][1])
data['lon'] = data['geom'].apply(lambda x: x.coords[0][0])

### How to plot polys in plotly with mapbox (only polys)

In [3]:
center = london_poly['geometry'][0].centroid
mapbox_token = 'pk.eyJ1IjoidGF6aXpvdiIsImEiOiJja2o1aTkyaXYweGt2MnZrM3pycW85bjFuIn0.o5E1GoUzcaw88zQuqae-AA'
px.set_mapbox_access_token(mapbox_token)

Attention! To plot from `geojson` need matching between `locations`
column values and `id` in every unit of `geojson` in `FeatureCollection`

In [5]:
fig = px.choropleth_mapbox(london_poly,
                           locations='id',
                           geojson=json.loads(london_poly.to_json()),
                           center = {'lat': center.y, 'lon': center.x})

Object `fig.update_layout` not found.


### Roads preprocessing

In [216]:
road_shapefiles = glob.glob('./data/oproad_essh_gb/data/*k.shp')

result = gpd.read_file(road_shapefiles[0])

for i in tqdm(range(1, len(road_shapefiles))):
    result = result.append(gpd.read_file(road_shapefiles[i]), ignore_index=True)

In [239]:
result = result.to_crs(epsg=4326)

result['is_in_london'] = result['geometry'].apply(lambda x: london_poly['geometry'][0].intersects(x))

london_roads_result = result[result['is_in_london']]

london_roads_result.info()

In [245]:
london_roads_result = london_roads_result.dropna(axis=1)

london_roads_result = london_roads_result.drop(columns=['startNode', 'endNode', 'function', 'identifier'])

london_roads_result = london_roads_result.reset_index(drop=True)

In [257]:
london_roads_result_js = london_roads_result.to_json()

with open('./data/london_roads.json', 'w') as fo:
    fo.write(json.dumps(london_roads_result_js))